In [ ]:
import sys
from pathlib import Path

# add directory with ribasim-repository
sys.path.append("..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping-repository
sys.path.append("..")
import ribasim_lumping

import pandas as pd
import geopandas as gpd

from numba.core.errors import NumbaDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Define base_dir, results_dir and network name
base_dir                = Path("..\\..\\ribasim_lumping_data\\")
network_name            = "tki_hooge_raam"
dhydro_basis_dir        = Path(base_dir, network_name, "2_dhydro\\basis\\")
dhydro_results_dir      = Path(base_dir, network_name, "2_dhydro\\results\\")
results_dir             = Path(base_dir, network_name, "3_ribasim\\network\\")

In [ ]:
# Create networkanalysis
network = ribasim_lumping.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    dhydro_basis_dir=dhydro_basis_dir,
    dhydro_results_dir=dhydro_results_dir,
    results_dir=results_dir,
    crs=28992
)
# network.export_to_geopackage()

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_dir               = Path(base_dir, network_name, "1_areas")
areas_file_name         = "afwateringseenheden2023_v1_HoogeRaam.shp"
areas_id_column         = "CODE"
network.read_areas(
    areas_file_path=Path(areas_dir, areas_file_name), 
    areas_id_column=areas_id_column
)

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
results = network.add_basis_simulation_data_set(
    set_name='winter', 
    simulation_name="HoogeRaamRibasim.dsproj_data"
)

In [ ]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_simulation_data_from_set(
    set_name="winter",
    simulations_names=[
        "0_01",
        "0_02",
        # "0_05",
        # "0_075",
        # "0_1",
        # "0_2",
        # "0_5",
        # "0_75",
        # "1_0",
        # "1_5",
        # "2_0",
    ],
    simulations_ts=[-1],
);

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=False,
    uniweirs=False,
    edges=False,
    structures_ids_to_include=[],
    structures_ids_to_exclude=["105BBW"],
    edge_ids_to_include=[],
    edge_ids_to_exclude=[],
);

In [ ]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.generate_ribasim_network()
# Export to geopackage
network.export_to_geopackage()

In [ ]:
# specify translation split_nodes to ribasim-nodes per type
split_node_type_conversion = {
    "weir": "TabulatedRatingCurve",
    "uniweir": "TabulatedRatingCurve",
    "pump": "Pump",
    "culvert": "TabulatedRatingCurve",
    "edge": "ManningResistance",
    "orifice": "TabulatedRatingCurve",
}
# specify translation for specific split_nodes to type of ribasim-node
split_node_id_conversion = {}

In [ ]:
# Generate Ribasim model and export to geopackage
ribasim_model = network.generate_ribasim_model(
    split_node_type_conversion=split_node_type_conversion,
    split_node_id_conversion=split_node_id_conversion,
)
ribasim_model.write(Path(results_dir, network.name))